# Political Labelling

This script determines the political affiliation (left, center, right) of each user in our sample by analyzing the retweets they have made.

We use a list of political influencers previously categorized as left, center, or right by La Silla Vacia, a Colombian news outlet. For each user, we tally the number of retweets they've made (excluding retweets with comments) that correspond to each influencer. From this data, we calculate the total number of tweets associated with each political category.

This process is carried out on tweets from the "Paro Nacional" period and on tweets that are not from this period, across three sections:

1. Paro Nacional tweets
2. Tweets not related to the Paro Nacional
3. Outputs

In [2]:
import pickle
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import scipy.sparse as sp
import os

path = r"/mnt/disk2/Data"

# Cargar Datos de congresistas y partidos

In [3]:
partidos = pd.read_excel(os.path.join(path,"clasificacion_partidos_v1.xlsx"),"Sheet1")
partidos = partidos.loc[:,['codigo_partido','ideologia']]

congresistas = pd.read_excel(os.path.join(path,"Twitter Congresistas.xlsx"),"Sheet1")
congresistas = congresistas.loc[:,('Partido', 'Twitter')]

# We load the tweets_lite DataFrame for the analysis
retweets = pd.read_pickle(os.path.join(path,"Tweets_DataFrames","retweets.gzip"), compression='gzip')

# We load the map that relates an ID to a political Label
with open(os.path.join(path,"Pickle","User_Dicts","mapa.pkl"), "rb") as file:
    mapa = pickle.load(file)

In [4]:
def partidos_dict(x):
    
    # Manualmente, reemplazar el nombre del partido por el ID
    d = {
        "PartidoSocialdeUnidadNacional":20050002,#Partido de la U
        "CentroDemocrático":20130001,#Centro Democrático
        "ListadelaDecencia":20180008,#Lista de la Decencia
        "AlianzaVerde":20090002,#Alianza Verde
        "PartidoConservadorColombiano":18490002,#Partido Conservador Colombiano
        "PartidoLiberalColombiano":18480001,#Partido Liberal Colombiano
        "ColombiaJustaLibres":20170001,#Colombia Justa Libres
        "PoloDemocráticoAlternativo":20050001,#Polo Democrático Alternativo
        "PartidoCambioRadical":20030001,#Partido Cambio Radical
        "ConsejoComunitariodeComunidadesNegrasPlayaRenaciente":20180014,#Consejo Comunitario Playa Renaciente
        "Comunes":20170003,#Partido Comunes
        "MovimientoAlternativoIndígenaySocial":20130002,#MAIS
        "ConsejoComunitarioLaMamuncia":20180029,#ConsejoComunitariolaMamuncia(wrongnameinexternaldatabase)
        "CoaliciónAlternativaSantandereana":20180042,#CoaliciónAlternativaSantandereana
        "MovimientoIndependientedeRenovaciónAbsoluta":20000036,#MIRA
        "OpciónCiudadana":20090001, #Partido Opción Ciudadana
    }
    
    # Si no encuentra el partido, entonces el ID por default es 0
    try:
        return d[x]
    except KeyError:
        return 0

def idelogias_dict(x):
    # Agrega la ideologia
    d = {
        1:'Izquierda',
        2:'Derecha',
        3:'Centro',
        4:'Sin Clasificar'
    }
    
    try:
        return d[x]
    # En caso de que el partio no haya sido encontrado por la anterior función, se deja esta bandera de aviso
    except KeyError:
        return 'nada'

In [5]:
# Diccionario con ID Nombre de twittero
user_name = (
    retweets[['Referenced Tweet Author ID', 'Referenced Tweet Author Name']]
    .drop_duplicates()
    .astype({'Referenced Tweet Author ID':'float64'})
    .set_index('Referenced Tweet Author ID')
    .to_dict()['Referenced Tweet Author Name']
)

# Limpiar nombres para agregar el ID de partido
congresistas.Partido = congresistas.Partido.str.replace(' ','')
congresistas['Partido ID'] = congresistas['Partido'].apply(partidos_dict)
congresistas_new = congresistas.merge(partidos,left_on='Partido ID', right_on='codigo_partido', how = 'left')

# Obetenemos la ideología
congresistas_new['Afiliacion'] = congresistas_new.ideologia.apply(idelogias_dict)

# Partidos que no aparecen en la base del CEDE
print("Partidos que no aparecen en la base del CEDE")
for p in congresistas_new[congresistas_new['Afiliacion']=='nada'].Partido.unique():
    print(p)

Partidos que no aparecen en la base del CEDE
AutoridadesIndígenasdeColombia
FuerzaAlternativaRevolucionariadelComún


In [6]:
# Dado que estos partidos no fueron, se agrega la ideologia a mano
# partido FARC y Autoridades Índigena vistos como izquierda

congresistas_new.loc[congresistas_new['Afiliacion']=='nada','Afiliacion'] = 'Izquierda'

# Pasamos datos importantes a un diccionario
mapa_2 = (
    congresistas_new.merge(retweets, left_on='Twitter', right_on='Referenced Tweet Author Name')
    .loc[:,('Referenced Tweet Author ID','Afiliacion')]
    .drop_duplicates()
    .astype({'Referenced Tweet Author ID':'float64'})
    .set_index('Referenced Tweet Author ID')
    .to_dict()['Afiliacion']
)

common_keys = list(set(mapa.keys()).intersection(set(mapa_2.keys())))
print(f"Hay {len(mapa)} usuarios identificados por la Silla")
print(f"Hay {len(mapa_2)} congresistas identificados según su partido")
print(f"Hay {len(common_keys)} usuarios identificados por ambas fuentes")
print('')
for key in common_keys:
    if mapa[key] != mapa_2[key]:
        # Usar el criterio de la silla vacía
        print(f"{user_name[key]}: Silla dice {mapa[key]} pero partido es {mapa_2[key]}")
    mapa_2.pop(key)

# Unir ambos diccioanrios
mapa_full = mapa_2 | mapa
print('')
print(f"Total de dicionarios {len(mapa_full)}")

# Guardar
with open(os.path.join(path,"Pickle","User_Dicts","mapa_full.pkl"), "wb") as file:
    pickle.dump(mapa_full, file)

Hay 93 usuarios identificados por la Silla
Hay 230 congresistas identificados según su partido
Hay 15 usuarios identificados por ambas fuentes

JERobledo: Silla dice Centro pero partido es Izquierda
RoyBarreras: Silla dice Izquierda pero partido es Derecha
angelamrobledo: Silla dice Centro pero partido es Izquierda
AABenedetti: Silla dice Izquierda pero partido es Derecha
intiasprilla: Silla dice Izquierda pero partido es Centro

Total de dicionarios 308


# CHECKPOINT: Cargar PRESAMPLE

Load all pickle files will need

In [9]:
# Cargar Tweets PRE SAMPLE Tweets (elecciones 2019 y enero 2020)
retweets = pd.read_pickle(os.path.join(path,"Tweets_DataFrames",'retweets_pre.gzip'), compression='gzip')

# Cargar mapa de Silla + Congresistas
with open(os.path.join(path,"Pickle","User_Dicts","mapa_full.pkl"), "rb") as file:
    mapa = pickle.load(file)

In [10]:
# A cada retweet, le asignamos una afiliaci´n según la persona retweteada
retweets["Party"] = retweets["Referenced Tweet Author ID"].map(mapa)

# Hay mucho NA's values 
print(retweets["Party"].value_counts(dropna=False).apply(lambda x: f"{x:,}"))

# solo se tendrán en cuenta los tweets TESTIMONIO
retweets = retweets[retweets["Party"].notna()]
print("")
print(f"Total de twitteros que retwitearon a alguien con clasificación {len(retweets['Author ID'].unique()):,}")
print("")
print(f"Total de Tweets TESTIMONIO (clasificados): {len(retweets):,.0f}")
retweets.head()

Party
NaN               5,670,001
Izquierda           578,799
Derecha             236,454
Centro              141,261
Sin Clasificar          537
Name: count, dtype: object

Total de twitteros que retwitearon a alguien con clasificación 24,749

Total de Tweets TESTIMONIO (clasificados): 957,051


,Tweet ID,Author ID,Author Name,Referenced Tweet Author ID,Referenced Tweet Author Name,Referenced Tweet ID,Date,Party
18,1.349944e+18,3.170157e+09,javiercruz2505,252402164.0,FisicoImpuro,1.349750e+18,2021/01/15 00:00:15,Izquierda
22,1.349944e+18,8.373459e+17,aarangob11,38670671.0,AABenedetti,1.349936e+18,2021/01/15 00:00:18,Izquierda
25,1.349944e+18,1.609187e+09,camilovegch,50981729.0,GustavoBolivar,1.349826e+18,2021/01/15 00:00:18,Izquierda
43,1.349945e+18,2.081536e+08,oscarjavierg,192941442.0,jflafaurie,1.349894e+18,2021/01/15 00:00:33,Derecha
45,1.349945e+18,1.261210e+08,SoyAlexys,53855557.0,CathyJuvinao,1.349929e+18,2021/01/15 00:00:33,Centro


We create a 4x1 positive integer vector for every tweeter in the community that registers the number of RTs that the user has based on the political affilation. 

In [11]:
# We create lambda-functions that count the number of RTs for each political label.
a = lambda x: np.sum(x == "Derecha")
b = lambda x: np.sum(x == "Izquierda")
c = lambda x: np.sum(x == "Centro")

# given per political label for each user using the lambda-functions.
rts_usuario_paro = retweets.groupby("Author ID").agg({"Party": [a,b,c]})

rts_usuario_paro.columns = ["Retweets Derecha",
                       "Retweets Izquierda",
                       "Retweets Centro"]

# Total RTs...
rts_usuario_paro["Retweets Totales"] = rts_usuario_paro.sum(axis=1)

rts_usuario_paro["Sin Clasificar"] = rts_usuario_paro['Retweets Totales'].apply(lambda x: 1 if x==0 else 0)

rts_usuario_paro.index = rts_usuario_paro.index.astype('float64')

# Now we determine the political affiliation by checking the index with the maximum.
rts_usuario_paro.sort_index()
print('Vector Database size is: ', rts_usuario_paro.shape)
rts_usuario_paro.head()

Vector Database size is:  (24749, 5)


,Retweets Derecha,Retweets Izquierda,Retweets Centro,Retweets Totales,Sin Clasificar
Author ID,,,,,
12996.0,0,10,7,17,0
784125.0,0,33,3,36,0
1061601.0,0,13,1,14,0
1981631.0,0,5,3,8,0
2331371.0,37,50,32,119,0


In [16]:
# Función de clasificación
def clasificar(row:pd.Series):
    # Revisar si segunda afiliación es igual a primera
    if row.nlargest(1).iloc[-1] == row.nlargest(2).iloc[-1]:
        return 'Sin Clasificar'
    else:
        return row.idxmax()

# Now we determine the political affiliation by checking the index with the maximum.
rts_usuario_paro["Afiliacion"] = rts_usuario_paro[["Retweets Centro", 
                                         "Retweets Derecha", 
                                         "Retweets Izquierda", 
                                         "Sin Clasificar"]].apply(clasificar,axis=1)

conditions = [
    (rts_usuario_paro['Afiliacion'] == 'Retweets Izquierda'),
    (rts_usuario_paro['Afiliacion'] == 'Retweets Derecha'),
    (rts_usuario_paro['Afiliacion'] == 'Retweets Centro'),
    (rts_usuario_paro['Afiliacion'] == 'Sin Clasificar')
]

choices = ['Izquierda', 'Derecha', 'Centro', 'Sin Clasificar']

rts_usuario_paro['Afiliacion'] = pd.Series(np.select(conditions, choices, default=''), index=rts_usuario_paro.index)

# We generate dummy variables for each political label...
rts_usuario_paro["Dummy Derecha"] = (rts_usuario_paro["Afiliacion"] == 'Derecha').astype('int32')
rts_usuario_paro["Dummy Izquierda"] = (rts_usuario_paro["Afiliacion"] == 'Izquierda').astype('int32')
rts_usuario_paro["Dummy Centro"] = (rts_usuario_paro["Afiliacion"] == 'Centro').astype('int32')
rts_usuario_paro["Dummy Sin Clasificar"] = (rts_usuario_paro["Afiliacion"] == 'Sin Clasificar').astype('int32')

# We see the sizes of our groups
print("Afiliaciones de usuarios en el PRE SAMPLE")
print(rts_usuario_paro['Afiliacion'].value_counts(dropna = False).apply(lambda x: f"{x:,.0f}"))
rts_usuario_paro.head()

Afiliaciones de usuarios en el PRE SAMPLE
Afiliacion
Izquierda         13,793
Derecha            5,326
Centro             4,331
Sin Clasificar     1,299
Name: count, dtype: object


,Retweets Derecha,Retweets Izquierda,Retweets Centro,Retweets Totales,Sin Clasificar,Afiliacion,Dummy Derecha,Dummy Izquierda,Dummy Centro,Dummy Sin Clasificar
Author ID,,,,,,,,,,
12996.0,0,10,7,17,0,Izquierda,0,1,0,0
784125.0,0,33,3,36,0,Izquierda,0,1,0,0
1061601.0,0,13,1,14,0,Izquierda,0,1,0,0
1981631.0,0,5,3,8,0,Izquierda,0,1,0,0
2331371.0,37,50,32,119,0,Izquierda,0,1,0,0


El problema es que en el codigo viejo, en esta línea de la segunda celda del CÓDIGO VIEJO no se estab reasignando el DataFrame.
```
tweets[tweets["Party"].notna()]
```
Por lo cual, todos aquellos twitteros que no retwittearon a alguien ya clasificado por el mapa o que simplemente no retwittearon, podian tener tweets cuya variable "Party" era nan

AL hacer groupby en rts_usuario_paro, esos usuarios parecia que no retwittearan a nadie. Por eso se clasificaban como Sin Clasificar en el **CODIGO VIEJO**

En el **CODIGO VIEJO** de una vez se utilizan retweets, más eficiente y si hace la reasignación. Por tanto, esos contribuyentes que no retwittearon a alguien clasificado o simplemente no retwittearon se eliminan. Los 18 contribuyentes en Sin Clasificar son por que Retwittearon a un Partido POlítica cuya clasificación ideologica según el CEDE era "Sin Clasificar"

Si estos no se eliminan

In [13]:
# Finally, we create a dictionary which stores the affiliation for each user.
user_to_party_paro = rts_usuario_paro.to_dict()['Afiliacion']

with open(os.path.join(path,"Pickle","User_Dicts","user_to_party_paro.pkl"), 'wb') as file:
    pickle.dump(user_to_party_paro,file)

rts_usuario_paro.to_pickle(os.path.join(path,"Pickle","User_Rts_Vector","rts_usuario_paro.pkl"))

# CHECKPOINT: Master Construction

In [21]:
# Información de usuarios del paro
users_information = pd.read_pickle(os.path.join(path, "Tweets_DataFrames","users_information.gzip"), compression='gzip').reset_index()

# vectores de clasificación para usuario PRESAMPLE
rts_usuario_paro = pd.read_pickle(os.path.join(path,"Pickle","User_Rts_Vector","rts_usuario_paro.pkl"))

# Resetar para hacer el merge
rts_usuario_paro = rts_usuario_paro.reset_index()
master = pd.merge(users_information,rts_usuario_paro,on='Author ID', how = 'outer',indicator=True)

# Aquellos
#master.loc[master['Afiliacion'].isna(),'Afiliacion'] = 'Sin Clasificar'
sin_afiliacion = sum(master['Afiliacion'].isna())
print(f"Usuarios del paro sin afiliación en PRE-SAMPLE {sin_afiliacion:,.0f}")
master = master.dropna(subset="Afiliacion")

master = (
    master.loc[:,('Author ID', 'Author Name', 'Afiliacion')]
    .rename(columns = {
        'Author ID':'User ID',
        'Author Name': 'Label',
        'Afiliacion': 'Political Affiliation'
    })
    .dropna()
    .drop_duplicates('User ID')
)

print(f"Usuarios en el Master {len(master):,.0f}")
print(f"Total de usuarios en el paro {len(master)+sin_afiliacion:,.0f}")
#master.to_csv(os.path.join(path,'Master.csv'),index=False)

Usuarios del paro sin afiliación en PRE-SAMPLE 12,598
Usuarios en el Master 24,746
Total de usuarios en el paro 37,344


In [22]:
ids_faltantes1 = set(retweets["Author ID"]) - set(user_to_party_paro.keys())
ids_faltantes2 = set(retweets["Referenced Tweet Author ID"]) - set(user_to_party_paro.keys())
ids_faltantes = np.concatenate((list(ids_faltantes1), list(ids_faltantes2)))
print(f"Faltan clasificar {len(np.unique(ids_faltantes)):,} usuarios")

# Los dejamos como inclasificados 
for usuario in ids_faltantes:
    user_to_party_paro[usuario] = np.nan

affilliation_df = pd.DataFrame(list(user_to_party_paro.items()), columns=['User ID', 'Political Affiliation'])
print(affilliation_df['Political Affiliation'].value_counts().apply(lambda x: f"{x:,}"))
print("\n"+"*"*100+"\n")
print(affilliation_df['Political Affiliation'].value_counts(normalize=True) * 100)

Faltan clasificar 154 usuarios
Political Affiliation
Izquierda         13,793
Derecha            5,326
Centro             4,331
Sin Clasificar     1,299
Name: count, dtype: object

****************************************************************************************************

Political Affiliation
Izquierda         55.731545
Derecha           21.520061
Centro            17.499697
Sin Clasificar     5.248697
Name: proportion, dtype: float64


# CHECKPOINT: Tweets not related to the Paro Nacional

Load all Pickle files needed

In [17]:
# We create an aux empty list to concatenate Tweets from January and October
aux = []

# We load January tweets
tweets_jan = pd.read_pickle(os.join.path(path,"Tweets_DataFrames",'tweets_jan21.gzip'), compression='gzip')

# We load October tweets
tweets_oct = pd.read_pickle(os.join.path(path,"Tweets_DataFrames",'tweets_oct19.gzip'), compression='gzip')

# Append both to the auxiliary list and concat them
aux.append(tweets_jan)
aux.append(tweets_oct)
tweets = pd.concat(aux)
print('October Shape: ', tweets_oct.shape)
print('January Shape: ', tweets_jan.shape)
print('Total Shape: ', tweets.shape)

October Shape:  (5424132, 25)
January Shape:  (5893802, 25)
Total Shape:  (11317934, 25)


In [18]:
# We load the map that relates an ID to a political Label
with open(os.path.join(path,"Pickle","User_Dicts","mapa_full.pkl"), "rb") as file:
    mapa = pickle.load(file)

In [19]:
# Now we assign each RT a political label according to its influencer's label.
retweets["Party"] = retweets["Referenced Tweet Author ID"].map(mapa)

# We select all non-NA labeled RT.
retweets = retweets[retweets["Party"].notna()]
print(retweets["Party"].value_counts())
retweets.head()

Party
Izquierda         3902002
Derecha           1091764
Centro             479654
Sin Clasificar       5105
Name: count, dtype: int64


,Tweet ID,Author ID,Author Name,Referenced Tweet Author ID,Referenced Tweet Author Name,Date,Referenced Tweet,Party
20,1405513539427635200,788250746,Laura_Milena98,142092456,JulianRoman,2021/06/17 08:11:55,1.405305e+18,Izquierda
22,1405510167844765696,788250746,Laura_Milena98,237445795,subcantante,2021/06/17 07:58:31,1.405317e+18,Izquierda
27,1404183860339003392,788250746,Laura_Milena98,237445795,subcantante,2021/06/13 16:08:15,1.404126e+18,Izquierda
29,1403881554112360448,788250746,Laura_Milena98,939908874357870592,DonIzquierdo_,2021/06/12 20:07:00,1.403795e+18,Izquierda
34,1403527396733640704,788250746,Laura_Milena98,237445795,subcantante,2021/06/11 20:39:42,1.403517e+18,Izquierda


In [21]:
# We create lambda-functions that count the number of RTs for each political 
# label.
a = lambda x: np.sum(x == "Derecha")
b = lambda x: np.sum(x == "Izquierda")
c = lambda x: np.sum(x == "Centro")

# given per political label for each user using the lambda-functions.
rts_usuario_jan_oct = retweets.groupby("Author ID").agg({"Party": [a,b,c]})

rts_usuario_jan_oct.columns = ["Retweets Derecha", 
                       "Retweets Izquierda", 
                       "Retweets Centro"]

# Total RTs...
rts_usuario_jan_oct["Retweets Totales"] = rts_usuario_jan_oct.sum(axis=1)
# We generate dummy variables for each political label...
rts_usuario_jan_oct["Dummy Derecha"] = (rts_usuario_jan_oct["Retweets Derecha"] != 0).astype('int32')
rts_usuario_jan_oct["Dummy Izquierda"] = (rts_usuario_jan_oct["Retweets Izquierda"] != 0).astype('int32')
rts_usuario_jan_oct["Dummy Centro"] = (rts_usuario_jan_oct["Retweets Centro"] != 0).astype('int32')

# Now we determine the political affiliation by checking the index with the maximum.
rts_usuario_jan_oct["Sin Clasificar"] = (rts_usuario_jan_oct["Retweets Totales"] == 0).astype('int32')
print('Vector Datbase size is: ',rts_usuario_jan_oct.shape)
rts_usuario_jan_oct.head()

Vector Datbase size is:  (33782, 8)


,Retweets Derecha,Retweets Izquierda,Retweets Centro,Retweets Totales,Dummy Derecha,Dummy Izquierda,Dummy Centro,Sin Clasificar
Author ID,,,,,,,,
12996,2,386,126,514,1,1,1,0
777978,1,1,1,3,1,1,1,0
784125,0,71,5,76,0,1,1,0
1061601,0,258,7,265,0,1,1,0
1981631,0,41,4,45,0,1,1,0


In [22]:
# Now we determine the political affiliation by checking the index with the maximum.
rts_usuario_jan_oct["Afiliacion"] = rts_usuario_jan_oct[["Retweets Centro", 
                                         "Retweets Derecha", 
                                         "Retweets Izquierda", 
                                         "Sin Clasificar"]].idxmax(axis=1)

rts_usuario_jan_oct['Afiliacion'].value_counts()

Afiliacion
Retweets Izquierda    23209
Retweets Derecha       7006
Retweets Centro        3562
Sin Clasificar            5
Name: count, dtype: int64

In [23]:
# Finally, we create a dictionary which stores the affiliation for each user.
user_to_party_jan_oct = {}

for index, row in rts_usuario_jan_oct.iterrows():
    author_id = int(index)
    afiliacion = row['Afiliacion']
    
    # Adding the author ID and affiliation to the dictionary
    user_to_party_jan_oct[author_id] = afiliacion

with open(os.path.join(path,"Pickle","user_to_party_jan_oct.pkl"), 'wb') as file:
    pickle.dump(user_to_party_jan_oct,file)

In [24]:
rts_usuario_jan_oct.to_pickle(os.path.join(path,"Pickle","User_Rts_Vector",'rts_usuario_jan_oct.pkl'))